IMPORTING LIBRARIES

In [1]:
import os
import shutil
import numpy as np
import networkx as nx
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document
from langchain_ollama.llms import OllamaLLM
from IPython.display import display, Markdown

MODEL CONFIGURATION

In [2]:
CHROMA_PATH = "chroma"
DATA_PATH = "./data"
OLLAMA_MODEL = "llama3.2"
COLLECTION_NAME = "ollama_vector_test"

def get_embedding_function():
    return HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

DOCUMENT PROCESSING

In [3]:
def load_documents():
    document_loader = PyPDFDirectoryLoader(DATA_PATH)
    return document_loader.load()

def split_documents(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=80,
        length_function=len,
        is_separator_regex=False,
    )
    return text_splitter.split_documents(documents)

def add_to_chroma(chunks: list[Document]):
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=get_embedding_function())
    
    existing_docs = db.get(include=['documents', 'metadatas'])
    
    if isinstance(existing_docs, dict) and "documents" in existing_docs:
        existing_texts = {doc.strip() for doc in existing_docs["documents"]}
    else:
        print("Data dari db.get() tidak sesuai format yang diharapkan.")
        return db
    
    print(f"📂 Jumlah dokumen dalam DB sebelum update: {len(existing_texts)}")
    
    new_chunks = []
    added_files = set()
    
    for chunk in chunks:
        chunk_text = chunk.page_content.strip()
        if chunk_text not in existing_texts:
            new_chunks.append(chunk)
            added_files.add((chunk.metadata.get('source'), chunk.metadata.get('page')))
    
    if new_chunks:
        print(f"📌 Menambahkan {len(new_chunks)} dokumen baru...")
        db.add_documents(new_chunks) 
        print(f"🔍 Jumlah dokumen dalam DB setelah persist: {len(db.get(include=['documents']).get('documents', []))}")
        print("📄 Dokumen yang baru ditambahkan:")
        for file, page in added_files:
            print(f"   - {file} (Page {page})")
    else:
        print("✅ Tidak ada dokumen baru untuk ditambahkan")
    
    return db

def clear_database():
    if os.path.exists(CHROMA_PATH):
        shutil.rmtree(CHROMA_PATH)
        print("🔥 Database telah dihapus.")

if __name__ == "__main__":
    #clear_database()
    documents = load_documents()
    chunks = split_documents(documents)
    db = add_to_chroma(chunks)

📂 Jumlah dokumen dalam DB sebelum update: 266
✅ Tidak ada dokumen baru untuk ditambahkan


CHECK DOCUMENTS IN DATABASE

In [7]:
print("🔍 Checking ChromaDB collection contents...")
results = db.get(include=["documents", "metadatas"])
print("📂 Documents retrieved:", len(results.get("documents", [])))
print("📄 Sample metadata:")
for meta in results.get("metadatas", [])[:10]: 
    print(f"- {meta}") 

🔍 Checking ChromaDB collection contents...
📂 Documents retrieved: 266
📄 Sample metadata:
- {'page': 0, 'page_label': '1', 'source': 'data\\22778-88667-1-PB.pdf'}
- {'page': 0, 'page_label': '1', 'source': 'data\\22778-88667-1-PB.pdf'}
- {'page': 0, 'page_label': '1', 'source': 'data\\22778-88667-1-PB.pdf'}
- {'page': 0, 'page_label': '1', 'source': 'data\\22778-88667-1-PB.pdf'}
- {'page': 1, 'page_label': '2', 'source': 'data\\22778-88667-1-PB.pdf'}
- {'page': 1, 'page_label': '2', 'source': 'data\\22778-88667-1-PB.pdf'}
- {'page': 1, 'page_label': '2', 'source': 'data\\22778-88667-1-PB.pdf'}
- {'page': 1, 'page_label': '2', 'source': 'data\\22778-88667-1-PB.pdf'}
- {'page': 1, 'page_label': '2', 'source': 'data\\22778-88667-1-PB.pdf'}
- {'page': 1, 'page_label': '2', 'source': 'data\\22778-88667-1-PB.pdf'}


TESTING MODELS WITH GRAPHRAG

In [8]:
def retrieve_summary(db, query, k=5):
    results = db.similarity_search_with_score(query, k=10) 
    if not results:
        return "❌ Tidak ada dokumen yang relevan ditemukan."
    
    results = sorted(results, key=lambda x: x[1], reverse=True)[:k]
    
    print("🔎 Dokumen yang digunakan untuk ringkasan:")
    for doc, score in results:
        print(f"- 📄 {doc.metadata['source']} - Page {doc.metadata['page']} (Score: {score:.2f})")
    
    return "\n".join([doc.page_content for doc, _ in results])

class GraphRAG:
    def __init__(self, chroma_db, model, prompt_template):
        self.db = chroma_db
        self.model = model
        self.prompt_template = prompt_template
        self.graph = nx.Graph()
        self.documents = []

    def fetch_documents_from_chroma(self):
        results = self.db.get(include=["documents", "metadatas"])
        documents = [(doc, meta) for doc, meta in zip(results.get("documents", []), results.get("metadatas", [])) if doc]
        return documents

    def generate_response(self, query):
        relevant_text = retrieve_summary(self.db, query, k=5)
        
        if "❌ Tidak ada dokumen yang relevan ditemukan." in relevant_text:
            return "Maaf, saya tidak dapat menemukan informasi yang relevan dalam dokumen yang ada."
        
        response_prompt = f"""
        Anda adalah asisten yang membantu menjawab pertanyaan berdasarkan dokumen.
        Tugas Anda adalah memberikan jawaban yang akurat dan terperinci. Jika perlu, gunakan analisis atau pemahaman tambahan.

        Berdasarkan teks berikut, jawab pertanyaan dengan akurat. Jika jawaban tidak lengkap, beri penjelasan lebih lanjut.

        Teks:
        {relevant_text}

        Pertanyaan:
        {query}
        """
    
        response = self.model.invoke(response_prompt)
        return response

if __name__ == "__main__":
    embedding_function = get_embedding_function()
    chroma_db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

    model = OllamaLLM(model=OLLAMA_MODEL)
    graph_rag = GraphRAG(chroma_db, model, None)

    query = "Apa faktor utama yang mempengaruhi kesehatan mental?"
    response = graph_rag.generate_response(query)
    display(Markdown(response))


🔎 Dokumen yang digunakan untuk ringkasan:
- 📄 data\ifransiska,+716-1511-1-PB.pdf - Page 2 (Score: 0.59)
- 📄 data\Kesehatan_Mental_Masyarakat_Indonesia_Pe.pdf - Page 4 (Score: 0.59)
- 📄 data\Kesehatan_Mental_Masyarakat_Indonesia_Pe.pdf - Page 4 (Score: 0.59)
- 📄 data\Kesehatan_Mental_Sumber_Daya_Manusia_Ind.pdf - Page 7 (Score: 0.59)
- 📄 data\Kesehatan_Mental_Sumber_Daya_Manusia_Ind.pdf - Page 0 (Score: 0.58)


Berdasarkan teks tersebut, faktor utama yang mempengaruhi kesehatan mental adalah dukungan masyarakat. Menurut teks, penderita gangguan kesehatan mental mengalami kesulitan dalam mendapatkan dukungan dari masyarakat karena mereka dianggap sebagai orang berbahaya atau pasien yang tidak dapat pulih. Oleh karena itu, memberikan edukasi tentang kesehatan mental dan gangguan kesehatan mental adalah penting untuk membantu masyarakat memahami kondisi penderita dan mengurangi stigma negatif terhadap mereka.

Selain dukungan masyarakat, teks juga menyebutkan bahwa faktor lain yang mempengaruhi kesehatan mental adalah wilayah. Penelitian empiris menunjukkan bahwa wilayah dengan kepadatan penduduk akan menghasilkan jumlah cacat mental yang lebih tinggi. Hal ini berarti bahwa faktor geografis dan demografi juga dapat mempengaruhi kesehatan mental.

Namun, perlu diingat bahwa teks tersebut tidak menyebutkan secara eksplisit tentang faktor utama yang mempengaruhi kesehatan mental secara umum. Namun, berdasarkan konteks dan isi teks, dapat disimpulkan bahwa dukungan masyarakat dan wilayah merupakan faktor-faktor yang signifikan dalam mempengaruhi kesehatan mental.

Dalam konteks ini, penelitian yang dilakukan oleh BPS tahun 2018 menunjukkan bahwa prevalensi cacat mental menyebar di seluruh wilayah Indonesia, termasuk Sulawesi. Hal ini dapat membantu memahami bahwa kesehatan mental bukan hanya dipengaruhi oleh faktor-faktor internal tetapi juga oleh faktor-faktor luar yang terkait dengan lingkungan dan masyarakat.

Dalam keseluruhan, teks tersebut menyebutkan bahwa kesehatan mental dipengaruhi oleh berbagai faktor, termasuk dukungan masyarakat, wilayah, dan lain-lain. Oleh karena itu, penting untuk memahami bahwa kesehatan mental bukan hanya dipengaruhi oleh faktor-faktor internal tetapi juga oleh faktor-faktor luar yang terkait dengan lingkungan dan masyarakat.